In [1]:
import pandas as pd

In [53]:
df_list = []
for i in range(1,4):
    df = pd.read_csv('df_save_final%d.csv' %i)
    col = list( df.columns )
    col[0] = 'date'
    df.columns = col
    df_list.append(df)

In [54]:

res = pd.DataFrame.merge(df_list[0], df_list[1], on = ['date'], how = 'outer')
res = pd.DataFrame.merge(res, df_list[2], on = ['date'], how = 'outer')

In [55]:
from datetime import datetime
import numpy as np
def str2date(string):
    date = datetime.strptime( string, "%Y/%m/%d")
    return date.date()

In [56]:
date_list = []
for date in res['date']:
    date_list.append(str2date(date))
res['date'] = np.array(date_list)

In [57]:
ret_df = res.sort_values(by=[ 'date' ],  ascending=True)
ret_df

,date,F0HKG05WWS:FO,F0HKG05WZ2:FO,F0HKG05WZ3:FO,F0HKG05WY6:FO,F0HKG05X2F:FO,F000010VSJ:FO,F00000ZNVB:FO,F00000ZD3C:FO,F00000WUPS:FO,...,F00000YE3R:FO,F00000YE3Y:FO,F00000YE3S:FO,F00000JW1C:FO,F00000YE41:FO,F00000YE3Z:FO,F00000YE42:FO,F00000YE3V:FO,F00000YE3W:FO,F00000YE3T:FO
0,2017-05-10,17.03,12.66,49.89,32.06,17.35,NaN,NaN,NaN,10.58,...,120.23,11.63,11.63,11.12,11.68,11.65,11.70,11.63,11.65,11.65
1,2017-05-11,16.99,12.67,49.97,32.09,17.34,NaN,NaN,NaN,10.58,...,120.99,11.70,11.70,11.18,11.75,11.72,11.77,11.70,11.72,11.72
2,2017-05-12,17.05,12.70,50.00,32.17,17.36,NaN,NaN,NaN,10.58,...,120.96,11.69,11.69,11.17,11.75,11.71,11.76,11.70,11.72,11.71
3,2017-05-15,17.16,12.87,50.59,32.60,17.57,NaN,NaN,NaN,10.61,...,121.65,11.76,11.76,11.24,11.81,11.77,11.83,11.76,11.78,11.77
4,2017-05-16,17.03,12.75,50.15,32.27,17.42,NaN,NaN,NaN,10.64,...,121.78,11.77,11.77,11.25,11.82,11.79,11.84,11.77,11.79,11.79
5,2017-05-17,17.03,12.74,50.15,32.12,17.39,NaN,NaN,NaN,10.60,...,120.44,11.64,11.64,11.13,11.70,11.66,11.71,11.64,11.66,11.66
6,2017-05-18,17.00,12.69,49.99,32.01,17.33,NaN,NaN,NaN,10.57,...,118.81,11.48,11.48,10.98,11.54,11.50,11.55,11.48,11.51,11.50
7,2017-05-19,16.92,12.58,49.69,31.79,17.21,NaN,NaN,NaN,10.61,...,120.07,11.60,11.60,11.09,11.65,11.62,11.67,11.60,11.63,11.62
8,2017-05-22,17.00,12.63,49.93,31.96,17.32,NaN,NaN,NaN,10.65,...,120.71,11.66,11.66,11.15,11.72,11.68,11.73,11.66,11.68,11.68
9,2017-05-23,17.06,12.74,50.29,32.12,17.35,NaN,NaN,NaN,10.66,...,120.65,11.65,11.65,11.14,11.71,11.67,11.73,11.66,11.68,11.67


In [58]:
import math
def stock_return(price_now, price_past):
    try:
        ret =  (price_now-price_past) / price_past
    except:
        ret = np.nan
    return ret

## weekly return

In [59]:
def get_weekly_return(a_df):
    date_list = a_df.iloc[:,0:1].values.tolist()
    price_list =a_df.iloc[:,1:2].values.tolist()
    
    return_list = []
    today = -1
    yesterday = -1
    this_week_last_price = 0
    last_week_last_price = price_list[0]
    
    tem_price_holer = []# 紀錄這周所有價格，為了判斷是否都是nan
    
    for date, price in zip(date_list, price_list):
        today = date[0].weekday()
        
        if today < yesterday: #由於monday 是0, sumday 是6 如果today的weekday比較小，代表換周了
            
            if len(tem_price_holer) != 0:
                this_week_last_price = tem_price_holer[-1]
                week_ret = stock_return(this_week_last_price, last_week_last_price)
                return_list.append(week_ret)
                
                last_week_last_price = this_week_last_price
            else:
                return_list.append(np.nan)
                last_week_last_price = np.nan
                
            tem_price_holer = [] #更新
            
        if not math.isnan(price[0]): #price 不是nan
            tem_price_holer.append(price[0])
        
        yesterday = today
        

    if len(tem_price_holer) != 0:
        this_week_last_price = tem_price_holer[-1]
        week_ret = stock_return(this_week_last_price, last_week_last_price)
        return_list.append(week_ret)
    else:
        return_list.append(np.nan)
        last_week_last_price = np.nan
    
    return return_list

In [60]:
id_list = list(ret_df.columns)

In [61]:
id_list[0:5]

['date', 'F0HKG05WWS:FO', 'F0HKG05WZ2:FO', 'F0HKG05WZ3:FO', 'F0HKG05WY6:FO']

In [62]:
len(id_list)

6193

In [63]:
data = {}
for i in range(1, 6193):
    ret_list = get_weekly_return(ret_df.iloc[:,[0,i]])
    
    #replace ret dataframe as weekly return
    a_id = id_list[i]
    data[a_id] = ret_list

In [64]:
df_week_ret = pd.DataFrame(data = data)

In [65]:
df_week_ret

,F0HKG05WWS:FO,F0HKG05WZ2:FO,F0HKG05WZ3:FO,F0HKG05WY6:FO,F0HKG05X2F:FO,F000010VSJ:FO,F00000ZNVB:FO,F00000ZD3C:FO,F00000WUPS:FO,F00000ZD39:FO,...,F00000YE3R:FO,F00000YE3Y:FO,F00000YE3S:FO,F00000JW1C:FO,F00000YE41:FO,F00000YE3Z:FO,F00000YE42:FO,F00000YE3V:FO,F00000YE3W:FO,F00000YE3T:FO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.007625,-0.009449,-0.006200,-0.011812,-0.008641,NaN,NaN,NaN,0.002836,NaN,...,-0.007358,-0.007699,-0.007699,-0.007162,-0.008511,-0.007686,-0.007653,-0.008547,-0.007679,-0.007686
2,0.014184,0.019078,0.021534,0.015728,0.012202,NaN,NaN,NaN,0.008483,NaN,...,0.016657,0.015517,0.015517,0.016231,0.016309,0.015491,0.016281,0.015517,0.015477,0.015491
3,0.013986,0.013261,0.008865,0.019511,0.019518,NaN,NaN,NaN,0.007477,NaN,...,0.000901,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.007471,0.012317,0.010154,0.002430,0.000000,NaN,NaN,NaN,-0.006494,NaN,...,0.004829,0.003396,0.003396,0.003549,0.003378,0.003390,0.003373,0.003396,0.003387,0.003390
5,-0.022818,-0.015970,-0.012179,-0.012424,-0.012950,NaN,NaN,NaN,0.000000,NaN,...,-0.008797,-0.010152,-0.010152,-0.008842,-0.009259,-0.009291,-0.009244,-0.009306,-0.010127,-0.009291
6,0.028021,0.021638,0.024266,0.024241,0.027952,NaN,NaN,NaN,0.000000,NaN,...,0.010847,0.009402,0.009402,0.009813,0.009346,0.009378,0.010178,0.009394,0.009378,0.009378
7,-0.006246,0.003026,0.005541,-0.000599,0.001110,NaN,NaN,NaN,-0.002801,NaN,...,0.001463,0.000847,0.000847,0.000883,0.000842,0.000845,0.000840,0.000000,0.000845,0.000845
8,-0.002857,-0.012821,-0.013490,-0.005695,-0.011086,NaN,NaN,NaN,-0.004682,NaN,...,-0.007306,-0.008460,-0.008460,-0.008826,-0.008410,-0.008439,-0.008389,-0.008460,-0.008439,-0.008439
9,0.041261,0.046600,0.041988,0.049442,0.047085,NaN,NaN,NaN,0.012230,NaN,...,0.042604,0.041809,0.041809,0.042743,0.042409,0.041702,0.041455,0.042662,0.041702,0.041702


## monthly return

In [66]:
def get_monthly_return(a_df):
    date_list = a_df.iloc[:,0:1].values.tolist()
    price_list =a_df.iloc[:,1:2].values.tolist()
    
    return_list = []
    today = -1
    yesterday = -1
    this_month_last_price = 0
    last_month_last_price = price_list[0]
    
    tem_price_holer = []# 紀錄這月所有價格，為了判斷是否都是nan
    
    for date, price in zip(date_list, price_list):
        today = date[0].day
        
        if today < yesterday: #由於1號 是1, 31號是31 如果today比較小，代表換月了
            
            if len(tem_price_holer) != 0:
                this_month_last_price = tem_price_holer[-1]
                month_ret = stock_return(this_month_last_price, last_month_last_price)
                return_list.append(month_ret)
                
                last_month_last_price = this_month_last_price
            else:
                return_list.append(np.nan)
                last_month_last_price = np.nan
                
            tem_price_holer = [] #更新
            
        if not math.isnan(price[0]): #price 不是nan
            tem_price_holer.append(price[0])
        
        yesterday = today

    
    return return_list

In [67]:
data2 = {}
for i in range(1, 6193):
    ret_list = get_monthly_return(ret_df.iloc[:,[0,i]])
    
    #replace ret dataframe as weekly return
    a_id = id_list[i]
    data2[a_id] = ret_list

In [68]:
df_month_ret = pd.DataFrame(data = data2)
df_month_ret

,F0HKG05WWS:FO,F0HKG05WZ2:FO,F0HKG05WZ3:FO,F0HKG05WY6:FO,F0HKG05X2F:FO,F000010VSJ:FO,F00000ZNVB:FO,F00000ZD3C:FO,F00000WUPS:FO,F00000ZD39:FO,...,F00000YE3R:FO,F00000YE3Y:FO,F00000YE3S:FO,F00000JW1C:FO,F00000YE41:FO,F00000YE3Z:FO,F00000YE42:FO,F00000YE3V:FO,F00000YE3W:FO,F00000YE3T:FO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.025791,0.037559,0.040324,0.036347,0.040369,NaN,NaN,NaN,-0.000935,NaN,...,0.018183,0.012853,0.012853,0.014324,0.013640,0.013687,0.014468,0.012853,0.012821,0.013687
2,0.083429,0.076923,0.062512,0.092026,0.081486,NaN,NaN,NaN,0.014981,NaN,...,0.059664,0.055838,0.055838,0.057370,0.057191,0.055696,0.056208,0.056684,0.056540,0.055696
3,0.055907,0.033613,0.029685,0.051606,0.045105,NaN,NaN,NaN,0.000923,NaN,...,0.026505,0.021635,0.021635,0.023372,0.022275,0.021583,0.023034,0.020817,0.021565,0.021583
4,0.012488,-0.007453,-0.011289,0.003915,0.001471,NaN,NaN,NaN,0.005530,NaN,...,0.018657,0.014118,0.014118,0.015498,0.014786,0.013302,0.014752,0.014118,0.014073,0.014867
5,0.054761,0.043686,0.039522,0.033541,0.045054,NaN,NaN,0.001998,0.009166,-0.001004,...,0.027766,0.024749,0.024749,0.026506,0.025307,0.025483,0.026014,0.024749,0.024672,0.024672
6,0.045837,0.033355,0.024502,0.053082,0.067948,NaN,NaN,0.000000,0.003633,-0.003015,...,0.019674,0.014340,0.015094,0.016432,0.015707,0.015060,0.016406,0.015094,0.015801,0.015049
7,-0.037120,-0.034810,-0.031338,-0.022934,-0.036419,NaN,-0.009036,-0.008973,0.004525,-0.007056,...,0.028801,0.025298,0.024535,0.026944,0.025773,0.024481,0.024945,0.024535,0.024444,0.024463
8,0.024617,0.022951,0.023327,0.022005,0.031421,NaN,-0.046606,-0.052314,0.023423,-0.020305,...,0.061969,0.057329,0.057329,0.059220,0.058148,0.057205,0.057981,0.057329,0.057122,0.057164
9,-0.035811,-0.037179,-0.034276,-0.033254,-0.046799,NaN,-0.026567,-0.025478,-0.020246,-0.017617,...,-0.039350,-0.041181,-0.041181,-0.039632,-0.040027,-0.042466,-0.041949,-0.041181,-0.042408,-0.042437
